<a href="https://colab.research.google.com/github/kristynpantoja/bsg_pytorch/blob/master/BSGcollab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math


from sklearn.datasets import fetch_20newsgroups
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import argparse
from types import SimpleNamespace

# import gensim.downloader as api
# from gensim.models import Word2Vec, FastText, KeyedVectors
# from os.path import isfile

import random
random.seed(1234)

import pandas as pd
import matplotlib.pyplot as plt

import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer


In [15]:
from nltk.tokenize import RegexpTokenizer

In [6]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kristyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kristyn/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# preprocessing stuff

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words): # preprocessing step
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
#     words = remove_stopwords(words)
    return words

In [17]:
# Get corpus: 20 news groups

newsgroups_train = fetch_20newsgroups(subset='train')

tokenizer = RegexpTokenizer(u'(?ui)\\b[a-z]{3,}\\b')

tokenized_corpus = [tokenizer.tokenize(document) for document in newsgroups_train.data]

# normalize_corpus = [normalize(document) for document in tokenized_corpus]

In [19]:
flattened = [word for document in tokenized_corpus for word in document]

In [20]:
vocab = set(flattened)
vocab = list(vocab)

In [21]:
len(vocab)

104685

In [ ]:
# # get vocabulary: with some preprocessing
# vectorizer = CountVectorizer(stop_words = 'english', 
#                              min_df=.01, max_df=0.9, 
#                              token_pattern = u'(?ui)\\b[a-z]{3,}\\b')

# count_vecs = vectorizer.fit_transform(newsgroups_train.data)

# # doc_term_matrix = count_vecs.toarray()
# # doc_term_matrix.shape # number of documents, number of words (in vocab)

# # note: vectorizer.get_feature_names() != vectorizer.vocabulary_

# vocabulary = vectorizer.get_feature_names()

In [ ]:
# # vocabulary = []
# # for sentence in tokenized_corpus:
# #     for token in sentence:
# #         if token not in vocabulary:
# #             vocabulary.append(token)

# word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
# idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

# vocabulary_size = len(vocabulary)


# window_size = 2
# idx_pairs = []
# # for each sentence
# for sentence in tokenized_corpus:
#     indices = [word2idx[word] for word in sentence]
#     # for each word, threated as center word
#     for center_word_pos in range(len(indices)):
#         # for each window position
#         for w in range(-window_size, window_size + 1):
#             context_word_pos = center_word_pos + w
#             # make soure not jump out sentence
#             if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
#                 continue
#             context_word_idx = indices[context_word_pos]
#             idx_pairs.append((indices[center_word_pos], context_word_idx))

# idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [ ]:
ragged_arrays = []
# for each document
for document in tokenized_corpus:
    ragged_array = []
    # for each word
    for word in document:
      ragged_arrays.append(word2idx[word])
    ragged_arrays.append(ragged_array)

In [ ]:
vectorizer.vocabulary_['lerxst']